In [30]:
%reset -f

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import time
import tools

In [31]:
# Parameters to tweak
lr = 1e-3
batchSize = 64
epochs = 30
loss_fn = nn.CrossEntropyLoss()

In [32]:
training_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

# Create 64 batches
train_dataloader = DataLoader(training_data, batch_size=batchSize)
test_dataloader = DataLoader(test_data, batch_size=batchSize)
len(test_dataloader)

157

In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [34]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Sigmoid(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [35]:
def train_data(model, optimizer):
    for xb, yb in train_dataloader:
        preds = model(xb)
        loss = loss_fn(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss = loss.item()
    print(f"Train loss: {loss:>7f}")

In [36]:
def test_data(model):
    num_batches = len(test_dataloader)
    size = len(test_dataloader.dataset)
    test_loss, corrects = 0, 0

    with torch.no_grad():
        for xb, yb in test_dataloader:
            preds = model(xb)
            test_loss += loss_fn(preds, yb).item()
            corrects += (preds.argmax(1) == yb).type(torch.float).sum().item()

    test_loss /= num_batches
    # test_loss = lo
    corrects /= size
    print(f"Test loss: \n Accuracy: {(100*corrects):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return corrects

In [37]:
epoch_pro = []
epoch_performance = []
for n in range(0,33):
    model = NeuralNetwork().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for t in range(epochs):
        start_time = time.time()

        train_data(model, optimizer)
        epoch_pro.append(test_data(model))
        epoch_performance.append(time.time()-start_time)
        

Train loss: 2.277558
Test loss: 
 Accuracy: 29.1%, Avg loss: 2.262814 

Train loss: 2.245372
Test loss: 
 Accuracy: 42.0%, Avg loss: 2.226364 

Train loss: 2.211341
Test loss: 
 Accuracy: 50.3%, Avg loss: 2.187842 

Train loss: 2.174446
Test loss: 
 Accuracy: 55.8%, Avg loss: 2.146092 

Train loss: 2.133755
Test loss: 
 Accuracy: 60.0%, Avg loss: 2.100130 

Train loss: 2.088460
Test loss: 
 Accuracy: 63.1%, Avg loss: 2.049203 

Train loss: 2.037933
Test loss: 
 Accuracy: 65.3%, Avg loss: 1.992869 

Train loss: 1.981807
Test loss: 
 Accuracy: 66.9%, Avg loss: 1.931091 

Train loss: 1.920056
Test loss: 
 Accuracy: 68.5%, Avg loss: 1.864291 

Train loss: 1.853046
Test loss: 
 Accuracy: 70.0%, Avg loss: 1.793343 

Train loss: 1.781552
Test loss: 
 Accuracy: 71.2%, Avg loss: 1.719482 

Train loss: 1.706702
Test loss: 
 Accuracy: 72.4%, Avg loss: 1.644151 

Train loss: 1.629858
Test loss: 
 Accuracy: 73.6%, Avg loss: 1.568825 

Train loss: 1.552471
Test loss: 
 Accuracy: 74.5%, Avg loss: 1.4

In [ ]:
tools.saveResult(epochs,lr,"crossentropyloss",batchSize,epoch_pro,epoch_performance, "model_cross_entropy_sigmoid")